In [15]:
%load_ext autoreload
%autoreload 2

import sys, pickle
sys.path.append('/home/andy/Documents/Research/software/basecode/')
from importscript import *

import tqdm
no_warnings()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
level=7; nside=2**level; divisor=2**35 * 4**(12-7)

query = """select source_id/{0} as hpx,
                  floor(phot_g_mean_mag/0.1)*0.1 as magbin,
                  count(*) as n,
                  count(*) filter (where parallax is not Null) as k
                from gaia_edr3.gaia_source
                group by hpx, magbin""".format(divisor)
%time _counts = sqlutilpy.get(query, asDict=True, **getdata.sql_args)

CPU times: user 25.9 s, sys: 4.25 s, total: 30.1 s
Wall time: 45min 16s


## Save data

In [29]:
magval = np.sort(np.unique(_counts['magbin']))
magval = magval[~np.isnan(magval)]

In [31]:
with h5py.File('/data/asfe2/Projects/astrometry/gaia3_astcounts.h', 'w') as hf:
    hf.create_dataset('magval', data=magval)
    for i, mag in enumerate(magval):
        hf.create_dataset(str(i)+'/n', data=_counts['n'][_counts['magbin']==mag])
        hf.create_dataset(str(i)+'/k', data=_counts['k'][_counts['magbin']==mag])
        hf.create_dataset(str(i)+'/hpx', data=_counts['hpx'][_counts['magbin']==mag])